# PIX and Brazil Payments Trends – Exploratory Analysis

This notebook performs initial exploration and cleaning of the Brazilian Payment Methods dataset (2016–2024).  
The goal is to prepare the data for SQL aggregation and Tableau visualization, using pandas and numpy, focusing on PIX and the decline of traditional payment methods (DOC, TED, Boleto, Checks).

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

## Load Dataset

In [2]:
df = pd.read_csv('../data/raw/brazilian_payment_methods_raw.csv') # Load Dataset

df.head(20) # Quick look at the data

,YearMonth,quantityPix,valuePix,quantityTED,valueTED,quantityTEC,valueTEC,quantityBankCheck,valueBankCheck,quantityBrazilianBoletoPayment,valueBrazilianBoletoPayment,quantityDOC,valueDOC
0,202405,5228826.97,2137205.79,69023.11,3482036.97,0.00,0.00,10467.85,40133.55,349180.92,506302.82,0.00,0.00
1,202404,4924681.00,2083746.68,70647.15,3652606.48,0.00,0.00,11179.22,43295.40,361633.28,524619.69,0.00,0.00
2,202403,4903392.34,1936188.21,67565.05,3333054.27,0.00,0.00,10122.46,36644.55,336435.46,469047.59,0.00,0.00
3,202402,4367133.78,1718217.65,65805.87,2981226.45,60.01,109.32,10373.38,37076.26,332269.47,463066.01,61.89,34.21
4,202401,4371541.99,1818473.97,70343.25,3337344.85,387.18,815.94,11919.12,41615.89,363550.32,516702.44,600.76,616.58
5,202312,4804459.10,1933836.67,80202.26,3629758.88,1152.37,2498.31,11241.91,39208.22,337552.05,484391.91,1412.03,1399.42
6,202311,4231283.46,1740874.02,74398.37,3135994.81,754.56,1711.45,11866.27,40735.16,345234.36,490503.75,1506.38,1521.77
7,202310,4069683.33,1666062.39,71870.52,3325168.12,664.92,1578.12,12902.42,46646.90,362721.59,512336.66,1746.38,1763.79
8,202309,3846749.03,1539042.27,69407.76,3190049.86,483.31,1085.19,11916.38,41653.43,340115.13,471176.87,2636.88,2532.14
9,202308,3767327.85,1529362.91,76572.58,3406864.68,614.09,1481.22,13113.98,46973.84,360865.30,501254.74,5253.95,3197.77


## Dataset Overview
Check column types, missing values, and basic statistics

In [3]:
df.info()
df.describe()
df.isna().sum() # Check for missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   YearMonth                       101 non-null    int64  
 1   quantityPix                     101 non-null    float64
 2   valuePix                        101 non-null    float64
 3   quantityTED                     101 non-null    float64
 4   valueTED                        101 non-null    float64
 5   quantityTEC                     101 non-null    float64
 6   valueTEC                        101 non-null    float64
 7   quantityBankCheck               101 non-null    float64
 8   valueBankCheck                  101 non-null    float64
 9   quantityBrazilianBoletoPayment  101 non-null    float64
 10  valueBrazilianBoletoPayment     101 non-null    float64
 11  quantityDOC                     101 non-null    float64
 12  valueDOC                        101 

YearMonth                         0
quantityPix                       0
valuePix                          0
quantityTED                       0
valueTED                          0
quantityTEC                       0
valueTEC                          0
quantityBankCheck                 0
valueBankCheck                    0
quantityBrazilianBoletoPayment    0
valueBrazilianBoletoPayment       0
quantityDOC                       0
valueDOC                          0
dtype: int64

> **Note:** 
Each number (0) tells us how many missing values there are in that column. 
0 means no missing data in that column — all rows have valid values. 
The dataset is complete for all columns.

## Convert YearMonth to datetime

In [4]:
df['YearMonth'] = pd.to_datetime(df['YearMonth'], format='%Y%m') # Convert YearMonth to datetime
df['year'] = df['YearMonth'].dt.year # Extract year for trend analysis
df['month'] = df['YearMonth'].dt.month  # Extract month for seasonal analysis

df.head()

,YearMonth,quantityPix,valuePix,quantityTED,valueTED,quantityTEC,valueTEC,quantityBankCheck,valueBankCheck,quantityBrazilianBoletoPayment,valueBrazilianBoletoPayment,quantityDOC,valueDOC,year,month
0,2024-05-01,5228826.97,2137205.79,69023.11,3482036.97,0.00,0.00,10467.85,40133.55,349180.92,506302.82,0.00,0.00,2024,5
1,2024-04-01,4924681.00,2083746.68,70647.15,3652606.48,0.00,0.00,11179.22,43295.40,361633.28,524619.69,0.00,0.00,2024,4
2,2024-03-01,4903392.34,1936188.21,67565.05,3333054.27,0.00,0.00,10122.46,36644.55,336435.46,469047.59,0.00,0.00,2024,3
3,2024-02-01,4367133.78,1718217.65,65805.87,2981226.45,60.01,109.32,10373.38,37076.26,332269.47,463066.01,61.89,34.21,2024,2
4,2024-01-01,4371541.99,1818473.97,70343.25,3337344.85,387.18,815.94,11919.12,41615.89,363550.32,516702.44,600.76,616.58,2024,1


The date was stored as **202405** and now it's **2024-05-01**.

> **Why?**  
> Converting `YearMonth` allows us to sort and plot data chronologically.  
> We also extract `year` and `month` for easier grouping later.

## Ensure Numeric Columns
These columns are numbers and need to be numeric for aggregation in SQL.

In [5]:
# Columns to convert to numeric (all except YearMonth)
numeric_cols = [
    'quantityPix', 'valuePix',
    'quantityTED', 'valueTED',
    'quantityTEC', 'valueTEC',
    'quantityBankCheck', 'valueBankCheck',
    'quantityBrazilianBoletoPayment', 'valueBrazilianBoletoPayment',
    'quantityDOC', 'valueDOC'
]

# Convert columns to numeric, coerce errors to NaN
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Verify the conversion
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   YearMonth                       101 non-null    datetime64[ns]
 1   quantityPix                     101 non-null    float64       
 2   valuePix                        101 non-null    float64       
 3   quantityTED                     101 non-null    float64       
 4   valueTED                        101 non-null    float64       
 5   quantityTEC                     101 non-null    float64       
 6   valueTEC                        101 non-null    float64       
 7   quantityBankCheck               101 non-null    float64       
 8   valueBankCheck                  101 non-null    float64       
 9   quantityBrazilianBoletoPayment  101 non-null    float64       
 10  valueBrazilianBoletoPayment     101 non-null    float64       
 11  quanti

YearMonth                         0
quantityPix                       0
valuePix                          0
quantityTED                       0
valueTED                          0
quantityTEC                       0
valueTEC                          0
quantityBankCheck                 0
valueBankCheck                    0
quantityBrazilianBoletoPayment    0
valueBrazilianBoletoPayment       0
quantityDOC                       0
valueDOC                          0
year                              0
month                             0
dtype: int64

## Create Metrics: Average Transaction Values
For each payment method (PIX, TED, TEC, DOC, Boleto, Check), the average value per transaction is calculated as:

- **avg_method = valueMethod / quantityMethod**

In [6]:
# Calculate average transaction values for each payment method
df['avg_pix'] = df['valuePix'] / df['quantityPix']
df['avg_ted'] = df['valueTED'] / df['quantityTED']
df['avg_doc'] = df['valueDOC'] / df['quantityDOC']
df['avg_check'] = df['valueBankCheck'] / df['quantityBankCheck']
df['avg_boleto'] = df['valueBrazilianBoletoPayment'] / df['quantityBrazilianBoletoPayment']

df[['YearMonth','avg_pix','avg_ted','avg_doc','avg_check','avg_boleto']].head(20)

,YearMonth,avg_pix,avg_ted,avg_doc,avg_check,avg_boleto
0,2024-05-01,0.408735,50.447408,NaN,3.833982,1.449973
1,2024-04-01,0.423123,51.702107,NaN,3.872846,1.450695
2,2024-03-01,0.394867,49.331041,NaN,3.620123,1.394168
3,2024-02-01,0.393443,45.303351,0.552755,3.574174,1.393646
4,2024-01-01,0.415980,47.443711,1.026333,3.491524,1.421268
5,2023-12-01,0.402509,45.257564,0.991070,3.487683,1.435014
6,2023-11-01,0.411429,42.151391,1.010217,3.432853,1.420785
7,2023-10-01,0.409384,46.266092,1.009969,3.615361,1.412479
8,2023-09-01,0.400089,45.960997,0.960279,3.495477,1.385345
9,2023-08-01,0.405954,44.491967,0.608641,3.581967,1.389036


> **Note:** Months with zero transactions (quantity = 0) will result in `NaN`.  
> This indicates that no transactions occurred that month, which is expected.

## Save Cleaned Dataset
Saving the cleaned dataset so it can be imported into SQL for aggregation and analysis:

In [7]:
df.to_csv('../data/cleaned/brazilian_payment_methods_clean.csv', index=False) # Save cleaned data to a new CSV file